# 環境を準備しよう

まず、Semantic Kernelと必要なパッケージをインストールしてバージョンを確認ます。

In [1]:
%%capture
%pip install -qU semantic-kernel python-dotenv

In [2]:
from semantic_kernel import __version__

__version__

'1.23.1'

次に、Azure OpenAI Serviceのキー情報を保存している「.env」ファイルを読み込みます。
`.env` ファイルには、次のような設定を記述します。

```bash
SK_AZURE_OPENAI_DEPLOYMENT_NAME=***
SK_AZURE_OPENAI_API_KEY=***
SK_AZURE_OPENAI_ENDPOINT=***
SK_AZURE_OPENAI_API_VERSION=***
```

この情報をコードで読み込みます。

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

# Function Callingを使って会話する

### Kernel（カーネル）のインスタンス作成

まずは、Semantic Kernel の中心となるクラス `Kernel` のインスタンスを作成します。

In [4]:
from semantic_kernel import Kernel

kernel = Kernel()

### AIサービス（Azure OpenAI Service）を設定する

Semantic Kernel では、さまざまな AI サービスを登録して使用できます。<br/>
ここでは Azure OpenAI Service の GPT-4o モデルを利用し、[Chat Completion API](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/how-to/chatgpt)を使う例を示します。

In [5]:
import os
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

service_id = None
chat_completion_service = AzureChatCompletion(
    deployment_name = os.getenv("SK_AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_key = os.getenv("SK_AZURE_OPENAI_API_KEY"),
    endpoint = os.getenv("SK_AZURE_OPENAI_ENDPOINT"), 
    api_version = os.getenv("SK_AZURE_OPENAI_API_VERSION"), 
    service_id = service_id,
)
kernel.add_service(chat_completion_service)

### チャット履歴を作成する

会話の履歴を扱うため、ChatHistory オブジェクトを作成します。

In [6]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()

### コアー関数を登録する

Function Calling の候補として使える コアー関数 を登録します。<br/>
今回は、Semantic Kernel が標準でサポートしている 2 つの関数（MathPlugin / TimePlugin）を例として使います。

In [7]:
from semantic_kernel.core_plugins import MathPlugin, TimePlugin

kernel.add_plugin(MathPlugin(), plugin_name="math")
kernel.add_plugin(TimePlugin(), plugin_name="time")

KernelPlugin(name='time', description=None, functions={'date': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='date', plugin_name='time', description='Get the current date.', parameters=[], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x00000163B007FFB0>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x00000163AEE40620>, method=<bound method TimePlugin.date of TimePlugin()>, stream_method=None), 'date_matching_last_day_name': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='date_matching_last_day_name', plugin_name='time', description='Get the date of the last day matching

### プロンプトテンプレート作成して登録する

今回のチャットでは、過去の会話履歴（chat_history）とユーザーの入力（request）を組み合わせるためのプロンプトを準備します。<br/>
"semantic-kernel" 形式のテンプレートで、両方の情報が埋め込まれるように設定します。

In [8]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

template = """
Previous information from chat:
{{$chat_history}}

User: {{$request}}
Assistant:
"""
prompt_template_config = PromptTemplateConfig(
    template=template,
    name="chat",
    description="Chat with the assistant",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="chat_history", description="The conversation history", is_required=False, default=""),
        InputVariable(name="request", description="The user's request", is_required=True),
    ],
    execution_settings=AzureChatPromptExecutionSettings(function_choice_behavior="auto"),
)

Kernel（カーネル）にプロンプトテンプレートを使って関数として登録します。

In [9]:
chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="ChatBot",
    prompt_template_config=prompt_template_config,
)

### 会話を開始し、AIの応答を受け取る

会話レベルでユーザーの入力を送信し、Chat Completion APIから応答を取得して出力します。<br/>
Kernel（カーネル）を呼び出す際は、Kernel Argumentsを使って引数を設定します。

ローカルの時間を出力してみます。

In [10]:
!echo %time%

16:28:27.09


Kernel（カーネル）に登録したコアー関数をFunction Callingによって呼び出されるように現在の時間を聞いてみます。

In [11]:
from semantic_kernel.functions import KernelArguments
user_input = "Hello, how are you? I'm Jin, What time is it in Japan?"
answer = await kernel.invoke(
    function=chat_function,
    arguments=KernelArguments(
        request=user_input,
        chat_history=chat_history,
    ),
)
print("User:", user_input)
chat_history.add_user_message(user_input)
print("Assistant:", answer)
chat_history.add_assistant_message(str(answer))

User: Hello, how are you? I'm Jin, What time is it in Japan?
Assistant: Hello Jin! I'm doing great, thank you. It's currently Saturday, March 8, 2025, 4:28 PM in Japan. How can I assist you further?


履歴の確認と、数値計算のコアー関数がFunction Callingで呼び出されるように聞いてみます。

In [12]:
user_input = "What is my name? And what is the result of 100.00000001+100.000000000002?"
answer = await kernel.invoke(
    function=chat_function,
    arguments=KernelArguments(
        request=user_input,
        chat_history=chat_history,
    ),
)
print("User:", user_input)
print("Assistant:", answer)

User: What is my name? And what is the result of 100.00000001+100.000000000002?
Assistant: Your name is Jin! 😊

As for the calculation:

\(100.00000001 + 100.000000000002 = 200.000000010002\)

Let me know if you'd like more help!
